# CV Optimizer & Cover Letter Generator - Code Explanation

This notebook explains how the CV Optimizer application code works. Each section is accompanied by detailed explanations to understand how the system functions.

## 📚 Notebook Structure

1. **Imports and Configuration** - Required libraries
2. **PDF Extraction** - How to extract text from PDFs
3. **Error Handling** - Parsing OpenAI errors
4. **CV Optimization** - The core optimization process
5. **Cover Letter Generation** - Creating natural-sounding letters
6. **Skills Matching** - Analyzing and comparing skills
7. **Flask Application** - API endpoints

---

**Note:** This notebook is for educational purposes. To run the full application, use python script: `app.py`


## 1. Imports and Libraries

Here are all the libraries needed to run the application:

- **LangChain**: Framework for building applications with LLMs
- **LangChain OpenAI**: Specific integration for OpenAI
- **PyPDF2 / pdfplumber**: Text extraction from PDFs
- **Flask**: Web framework for creating the API
- **Typing**: For Python type hints (better readability)


In [ ]:
# Main imports
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from typing import Dict, Any, Optional, List
import re
import json
from io import BytesIO

# For PDF extraction
import PyPDF2
import pdfplumber

# For Flask (if you want to test endpoints)
# from flask import Flask, request, jsonify
# from flask_cors import CORS

print("✅ All libraries imported successfully!")


## 2. PDF Text Extraction

### How it works?

PDF extraction uses **two methods** to maximize success chances:

1. **pdfplumber** (primary method): Better for complex PDFs with layouts
2. **PyPDF2** (fallback method): More basic but more robust

### Why two methods?

Some PDFs are better read by one method than the other. By using both, we maximize the chances of correctly extracting text, even from poorly formatted PDFs.


In [ ]:
def extract_text_from_pdf(pdf_file) -> str:
    """
    Extract text from a PDF file using multiple methods for better accuracy.
    
    Strategy: Try pdfplumber first (better for complex layouts),
    then fallback to PyPDF2 if it fails.
    """
    text = ""
    
    # Method 1: pdfplumber (better for complex layouts)
    try:
        # Convert bytes to BytesIO if necessary
        if isinstance(pdf_file, bytes):
            pdf_file = BytesIO(pdf_file)
        
        # Open PDF and extract text page by page
        with pdfplumber.open(pdf_file) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
        
        # If we successfully extracted text, return it
        if text.strip():
            return text.strip()
    except Exception as e:
        print(f"⚠️ pdfplumber failed: {e}")
        # Continue with fallback method
    
    # Method 2: PyPDF2 (fallback)
    try:
        # Reset file if it was bytes
        if isinstance(pdf_file, bytes):
            pdf_file = BytesIO(pdf_file)
        
        # Read PDF with PyPDF2
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    except Exception as e:
        print(f"⚠️ PyPDF2 failed: {e}")
    
    # Return extracted text (may be empty if both methods failed)
    return text.strip() if text else ""

## 3. OpenAI Error Handling

### Why is it important?

OpenAI API errors are often technical and hard to understand for users. This function **parses errors** and transforms them into clear messages with instructions to solve the problem.

### Types of errors handled:

- **401**: Invalid API key → Instructions to get a new key
- **429**: Rate limit → Informative message
- **500**: Server error → Informative message
- **Insufficient credits**: Link to billing page
- **Invalid model**: Suggestion to choose another model


In [ ]:
def parse_openai_error(error: Exception) -> Dict[str, Any]:
    """
    Parse OpenAI API errors and return user-friendly messages.
    
    This function analyzes the error message and identifies the error type
    to return a clear and actionable message to the user.
    """
    error_str = str(error)
    
    # Initialize default values
    error_code = None
    error_message = error_str
    user_message = "An error occurred when calling the OpenAI API."
    
    # Detect 401 error (Invalid API Key)
    if "401" in error_str or "invalid_api_key" in error_str.lower() or "Incorrect API key" in error_str:
        error_code = 401
        user_message = (
            "Error: Invalid OpenAI API key.\n\n"
            "Your API key is not valid or has expired. Please:\n"
            "1. Verify that you copied the complete key\n"
            "2. Get a new key at: https://platform.openai.com/account/api-keys\n"
            "3. Verify that your OpenAI account has available credits"
        )
    
    # Detect 429 error (Rate Limit)
    elif "429" in error_str or "rate_limit" in error_str.lower():
        error_code = 429
        user_message = (
            "Error: Rate limit exceeded.\n\n"
            "You have made too many requests. Please wait a few moments before trying again."
        )
    
    # Detect 500 error (Server Error)
    elif "500" in error_str or "internal_error" in error_str.lower():
        error_code = 500
        user_message = (
            "Error: OpenAI server problem.\n\n"
            "The OpenAI server is experiencing difficulties. Please try again in a few moments."
        )
    
    # Detect insufficient credits
    elif "insufficient_quota" in error_str.lower() or "billing" in error_str.lower():
        error_code = "billing"
        user_message = (
            "Error: Insufficient credits.\n\n"
            "Your OpenAI account no longer has available credits. "
            "Please add credits at: https://platform.openai.com/account/billing"
        )
    
    # Detect invalid model
    elif "model" in error_str.lower() and ("not found" in error_str.lower() or "invalid" in error_str.lower()):
        error_code = "model"
        user_message = (
            "Error: Invalid model.\n\n"
            "The selected model is not available. Please choose another model."
        )
    
    return {
        "error_code": error_code,
        "error_message": error_message,
        "user_message": user_message
    }

## 4. CV Optimization

### How it works

CV optimization uses **LangChain** to create a processing chain:

1. **Create the LLM model**: Configure the OpenAI model (GPT-4o-mini by default)
2. **Build the prompt**: Create a system prompt that guides the AI
3. **Invoke**: Send the CV and job description to the AI
4. **Result**: The AI returns an optimized CV

### Important parameters

- **Temperature (0.3)**: Low for a deterministic and consistent CV
- **Language**: The prompt is adapted according to the chosen language
- **Filters**: Min/max number of experiences, max years


In [ ]:
def optimize_cv(
    cv_text: str,
    job_description: str,
    api_key: str,
    model: str = "gpt-4o-mini",
    temperature: float = 0.3,
    min_experiences: int = 3,
    max_experiences: int = 8,
    max_date_years: Optional[int] = None,
    language: str = "fr",
) -> Dict[str, Any]:
    """
    Optimize a CV for a specific job description.
    
    This function uses LangChain to create a processing chain
    that optimizes the CV according to the job description.
    """
    
    # Step 1: Initialize LangChain model with OpenAI
    llm = ChatOpenAI(
        model=model,
        temperature=temperature,  # Low temperature = more deterministic
        api_key=api_key
    )
    
    # Step 2: Build dynamic filtering instructions
    date_filter = ""
    if max_date_years:
        date_filter = f"\n- Only include experiences from the last {max_date_years} years"
    
    exp_filter = f"\n- Include between {min_experiences} and {max_experiences} professional experiences"
    
    # Step 3: Map language for the prompt
    language_names = {
        "fr": "French (Français)",
        "en": "English",
        "es": "Spanish (Español)"
    }
    target_language = language_names.get(language, "French (Français)")
    
    # Step 4: Build system message (instructions for AI)
    system_message = f"""You are an expert CV/resume optimizer. Your task is to tailor a candidate's CV to match a specific job description while maintaining authenticity and truthfulness.

CRITICAL: The entire CV must be written in {target_language}. All sections, descriptions, and content must be in this language.

Guidelines:
- Keep all information factual and accurate
- Reorganize and rephrase content to highlight relevant skills and experiences
- Use action verbs and quantify achievements where possible
- Maintain professional formatting with clear sections
- Ensure ATS (Applicant Tracking System) compatibility
- Keep the same structure: Header, Summary, Experience, Education, Skills, etc.{date_filter}{exp_filter}
- Focus on experiences and skills most relevant to the job
- Remove or de-emphasize irrelevant information
- Use industry-standard terminology from the job description where appropriate
- Write everything in {target_language} - section headers, descriptions, and all text"""
    
    # Step 5: Create prompt template with LangChain
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_message),  # General instructions
        ("human", """Job Description:
{job_description}

Original CV:
{cv_text}

Create an optimized CV tailored to this job description. Maintain all factual information but reorganize and rephrase to maximize relevance and impact.""")
    ])
    
    # Step 6: Create the chain (prompt → LLM)
    chain = prompt | llm
    
    # Step 7: Execute the chain with data
    try:
        response = chain.invoke({
            "job_description": job_description,
            "cv_text": cv_text
        })
        
        optimized_cv = response.content
        
        return {
            "optimized_cv": optimized_cv,
            "model_used": model,
            "temperature": temperature,
            "word_count": len(optimized_cv.split())
        }
    except Exception as e:
        # Handle errors with our parsing function
        error_info = parse_openai_error(e)
        return {
            "error": error_info["user_message"],
            "error_code": error_info["error_code"],
            "error_details": error_info["error_message"],
            "optimized_cv": None
        }

print("OK: CV optimization function defined!")


## 5. Cover Letter Generation

### Goal: Avoid AI Detection

Unlike CV optimization (which must be factual), cover letters must **sound natural and human**.

### Strategies used:

1. **Higher temperature (0.7)**: More creativity and variety
2. **Anti-AI instructions**: The prompt guides the AI to avoid detectable patterns
3. **Language-specific conventions**: Adapted politeness formulas (Madame/Monsieur in French, Dear in English, etc.)
4. **Post-generation cleaning**: Removal of phrases that are too "AI-like"


In [ ]:
def generate_cover_letter(
    cv_text: str,
    optimized_cv: str,
    job_description: str,
    api_key: str,
    model: str = "gpt-4o-mini",
    temperature: float = 0.7,  # Higher than for CV!
    target_words: int = 300,
    language: str = "fr",
) -> Dict[str, Any]:
    """
    Generate a personalized cover letter that doesn't sound AI-generated.
    
    Temperature is higher (0.7) to allow more creativity
    and avoid repetitive patterns typical of AI.
    """
    
    # Initialize the model
    llm = ChatOpenAI(
        model=model,
        temperature=temperature,  # More creative than for CV
        api_key=api_key
    )
    
    # Round word count to nearest 10
    target_words = round(target_words / 10) * 10
    
    # Map language
    language_names = {
        "fr": "French (Français)",
        "en": "English",
        "es": "Spanish (Español)"
    }
    target_language = language_names.get(language, "French (Français)")
    
    # Language-specific guidelines for letter conventions
    language_guidelines = {
        "fr": "- Use appropriate French business letter conventions\n- Use 'Madame, Monsieur' or 'Madame, Monsieur le Directeur' for formal openings\n- Use 'Cordialement' or 'Bien cordialement' for closings\n- Avoid anglicisms and use proper French expressions",
        "en": "- Use appropriate English business letter conventions\n- Use 'Dear [Name]' or 'Dear Hiring Manager' for openings\n- Use 'Sincerely' or 'Best regards' for closings\n- Use natural English expressions and idioms",
        "es": "- Use appropriate Spanish business letter conventions\n- Use 'Estimado/a [Nombre]' or 'A quien corresponda' for openings\n- Use 'Atentamente' or 'Saludos cordiales' for closings\n- Use natural Spanish expressions and idioms"
    }
    lang_guidelines = language_guidelines.get(language, language_guidelines["fr"])
    
    # System message with anti-AI detection instructions
    system_message = f"""You are a professional writer helping someone write a cover letter. Your goal is to create a letter that sounds completely natural and human-written, NOT AI-generated.

CRITICAL: The entire cover letter must be written in {target_language}. All content, greetings, and closings must be in this language.

CRITICAL GUIDELINES TO AVOID AI DETECTION:
- Use varied sentence lengths (mix short and long sentences)
- Include occasional conversational elements and personal touches
- Avoid overly formal or robotic language
- Use natural transitions, not formulaic connectors
- Include specific details and anecdotes when possible
- Vary your vocabulary - don't repeat the same phrases
- Write in a warm, professional but authentic tone
- Avoid clichés - be more direct and personal
- Use contractions naturally where appropriate (if culturally acceptable in {target_language})
- Include minor imperfections that make it feel human (but keep it professional)
- Don't overuse exclamation points or excessive enthusiasm
- Be specific about why THIS company and THIS role, not generic statements
{lang_guidelines}

Target length: approximately {target_words} words (±20 words is acceptable).

Structure:
- Opening: Engaging, specific hook (not generic openings)
- Body paragraphs: 2-3 paragraphs connecting experience to role
- Closing: Professional but warm sign-off appropriate for {target_language}

Make it sound like a real person wrote this in {target_language}, not an AI assistant."""
    
    # Create the prompt
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_message),
        ("human", """Job Description:
{job_description}

Candidate's Original CV:
{cv_text}

Candidate's Optimized CV (for reference):
{optimized_cv}

Write a compelling, natural-sounding cover letter in {target_language} that:
1. Shows genuine interest in this specific role and company
2. Highlights relevant experience from the CV
3. Connects the candidate's background to the job requirements
4. Sounds completely human-written (no AI patterns)
5. Is approximately {target_words} words
6. Uses appropriate business letter conventions for {target_language}

Make it personal, authentic, and engaging while remaining professional. Write everything in {target_language}.""")
    ])
    
    # Create and execute the chain
    chain = prompt | llm
    
    try:
        response = chain.invoke({
            "job_description": job_description,
            "cv_text": cv_text,
            "optimized_cv": optimized_cv,
            "target_words": target_words,
            "target_language": target_language
        })
        
        cover_letter = response.content.strip()
        
        # Clean obvious AI artifacts
        cover_letter = clean_ai_artifacts(cover_letter)
        
        word_count = len(cover_letter.split())
        
        return {
            "cover_letter": cover_letter,
            "word_count": word_count,
            "target_words": target_words,
            "model_used": model,
            "temperature": temperature
        }
    except Exception as e:
        error_info = parse_openai_error(e)
        return {
            "error": error_info["user_message"],
            "error_code": error_info["error_code"],
            "error_details": error_info["error_message"],
            "cover_letter": None
        }


def clean_ai_artifacts(text: str) -> str:
    """
    Remove obvious AI-generated patterns from text.
    
    This function cleans the text to remove phrases
    that are too typical of AI generations.
    """
    # Common phrases generated by AI
    ai_phrases = [
        r"I am writing to express my (?:sincere |genuine )?interest",
        r"I am excited to (?:apply|submit my application)",
        r"Thank you for (?:considering|taking the time)",
        r"I believe I would be a (?:great|excellent|perfect) fit",
    ]
    
    # Remove these phrases
    for phrase in ai_phrases:
        text = re.sub(phrase, "", text, flags=re.IGNORECASE)
    
    # Replace overly formal formulations
    text = re.sub(r"\bI am confident that\b", "I'm confident", text, flags=re.IGNORECASE)
    text = re.sub(r"\bI would like to\b", "I'd like to", text, flags=re.IGNORECASE)
    
    # Clean up multiple spaces
    text = re.sub(r" +", " ", text)
    text = re.sub(r"\n\s*\n\s*\n", "\n\n", text)
    
    return text.strip()

print("OK: Cover letter generation function defined!")


## 6. Skills Extraction and Matching

### Two-step process

1. **Extraction**: Uses AI to extract skills from CV and job offer
2. **Matching**: Compares skills and identifies:
   - Matches (green)
   - Missing skills (red)
   - Skills only in CV (gray)
   - Interesting CV skills (blue) - identified by AI

### Why use AI for extraction?

Manual extraction would be tedious. AI understands context and can identify:
- Technical skills (Python, React, etc.)
- Soft skills (leadership, communication, etc.)
- Domain expertise (marketing, finance, etc.)
- Certifications and languages


In [ ]:
def extract_skills(text: str, api_key: str, text_type: str = "cv", model: str = "gpt-4o-mini") -> Dict[str, Any]:
    """
    Extract skills from CV or job description using AI.
    
    The AI analyzes the text and extracts the main skills.
    We use a low temperature (0.2) for a deterministic result.
    """
    llm = ChatOpenAI(
        model=model,
        temperature=0.2,  # Very low for consistent results
        api_key=api_key
    )
    
    # Different prompts depending on text type
    if text_type == "cv":
        system_message = """You are an expert at analyzing CVs and resumes. Extract the main skills, competencies, and technical abilities from the CV.

Return ONLY a JSON array of skills, nothing else. Each skill should be a short, clear term (2-4 words max).
Focus on:
- Technical skills (programming languages, tools, software)
- Soft skills (communication, leadership, etc.)
- Domain expertise (marketing, finance, etc.)
- Certifications and qualifications
- Languages

Format: ["skill1", "skill2", "skill3", ...]"""
        
        prompt_text = f"""Extract all the main skills and competencies from this CV:

{text}

Return a JSON array of skills only, no explanations."""
    else:
        system_message = """You are an expert at analyzing job descriptions. Extract the required and preferred skills, competencies, and qualifications from the job description.

Return ONLY a JSON array of skills, nothing else. Each skill should be a short, clear term (2-4 words max).
Focus on:
- Required technical skills
- Preferred technical skills
- Soft skills mentioned
- Domain expertise required
- Certifications or qualifications needed
- Language requirements

Format: ["skill1", "skill2", "skill3", ...]"""
        
        prompt_text = f"""Extract all the required and preferred skills from this job description:

{text}

Return a JSON array of skills only, no explanations."""
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_message),
        ("human", prompt_text)
    ])
    
    chain = prompt | llm
    
    try:
        response = chain.invoke({})
        content = response.content.strip()
        
        # Clean content (remove markdown if present)
        content = re.sub(r'```json\s*', '', content)
        content = re.sub(r'```\s*', '', content)
        content = content.strip()
        
        # Parse JSON
        try:
            skills = json.loads(content)
            if not isinstance(skills, list):
                skills = [skills]
        except json.JSONDecodeError:
            # Fallback: try to extract array from text
            match = re.search(r'\[(.*?)\]', content, re.DOTALL)
            if match:
                array_content = match.group(1)
                skills = [s.strip().strip('"\'') for s in array_content.split(',')]
                skills = [s for s in skills if s]
            else:
                # Last resort: split by commas
                skills = [s.strip().strip('"\'') for s in content.replace('\n', ',').split(',')]
                skills = [s for s in skills if s and len(s) > 1]
        
        # Clean and normalize
        skills = [skill.strip() for skill in skills if skill and len(skill.strip()) > 1]
        skills = list(set(skills))  # Remove duplicates
        skills = sorted(skills)  # Sort alphabetically
        
        return {
            "skills": skills,
            "count": len(skills),
            "text_type": text_type
        }
    except Exception as e:
        error_info = parse_openai_error(e)
        return {
            "error": error_info["user_message"],
            "error_code": error_info["error_code"],
            "skills": [],
            "count": 0
        }

print("OK: Skills extraction function defined!")


### Skills Matching

Matching compares skill lists and categorizes them:

1. **Exact/partial matches**: Skills present in both
2. **CV only**: Skills from CV not mentioned in the offer
3. **Offer only**: Required skills missing from CV
4. **Interesting**: CV skills that AI deems useful for the position (even if not mentioned)

The "interesting" part uses AI to analyze context and identify skills that add value.


In [ ]:
def match_skills(cv_skills: List[str], job_skills: List[str], cv_text: str = "", job_text: str = "", api_key: str = "", model: str = "gpt-4o-mini") -> Dict[str, Any]:
    """
    Match skills between CV and job description.
    
    This function compares skills and uses AI to identify
    which CV skills are "interesting" for the position.
    """
    # Normalize skills for comparison (lowercase, no spaces)
    cv_normalized = {skill.lower().strip(): skill for skill in cv_skills}
    job_normalized = {skill.lower().strip(): skill for skill in job_skills}
    
    # Find matches
    matched = []
    cv_only = []
    job_only = []
    
    # Check for matches (exact or partial)
    for cv_key, cv_original in cv_normalized.items():
        found_match = False
        for job_key, job_original in job_normalized.items():
            # Exact match
            if cv_key == job_key:
                matched.append({
                    "cv_skill": cv_original,
                    "job_skill": job_original,
                    "match_type": "exact"
                })
                found_match = True
                break
            # Partial match (one contains the other)
            elif cv_key in job_key or job_key in cv_key:
                matched.append({
                    "cv_skill": cv_original,
                    "job_skill": job_original,
                    "match_type": "partial"
                })
                found_match = True
                break
        
        if not found_match:
            cv_only.append(cv_original)
    
    # Find job skills not in CV
    for job_key, job_original in job_normalized.items():
        found_in_cv = False
        for cv_key in cv_normalized.keys():
            if cv_key == job_key or cv_key in job_key or job_key in cv_key:
                found_in_cv = True
                break
        if not found_in_cv:
            job_only.append(job_original)
    
    # Identify "interesting" skills with AI
    interesting = []
    if cv_only and api_key and cv_text and job_text:
        try:
            llm = ChatOpenAI(
                model=model,
                temperature=0.3,
                api_key=api_key
            )
            
            # Prompt to identify interesting skills
            prompt_text = f"""Analyze which CV skills from the list below would be valuable or interesting for this job, even though they are not explicitly mentioned in the job description.

Job Description (excerpt):
{job_text[:1000]}

CV Skills that are NOT in the job description:
{', '.join(cv_only[:20])}

Return ONLY a JSON array of skills from the list above that would be valuable for this job. Return an empty array [] if none are relevant.

Format: ["skill1", "skill2", ...]"""
            
            prompt = ChatPromptTemplate.from_messages([
                ("system", "You are an expert at matching candidate skills to job requirements. Identify skills that would add value even if not explicitly required."),
                ("human", prompt_text)
            ])
            
            chain = prompt | llm
            response = chain.invoke({})
            content = response.content.strip()
            
            # Parse response
            content = re.sub(r'```json\s*', '', content)
            content = re.sub(r'```\s*', '', content)
            content = content.strip()
            
            try:
                interesting_parsed = json.loads(content)
                if isinstance(interesting_parsed, list):
                    # Match with original skills
                    interesting_normalized = [s.lower().strip() for s in interesting_parsed]
                    for skill in cv_only:
                        if skill.lower().strip() in interesting_normalized:
                            interesting.append(skill)
            except json.JSONDecodeError:
                # Fallback: all CV-only skills
                interesting = cv_only.copy()
        except Exception:
            # If AI analysis fails, use all CV-only skills
            interesting = cv_only.copy()
    else:
        # If no API key, consider all CV-only skills as interesting
        interesting = cv_only.copy()
    
    return {
        "matched": [m["cv_skill"] for m in matched],  # Green: in both
        "cv_only": [s for s in cv_only if s not in interesting],  # Gray: CV only
        "job_only": job_only,  # Red: missing
        "interesting": interesting,  # Blue: interesting
        "stats": {
            "total_cv": len(cv_skills),
            "total_job": len(job_skills),
            "matched_count": len(matched),
            "missing_count": len(job_only),
            "cv_only_count": len([s for s in cv_only if s not in interesting]),
            "interesting_count": len(interesting),
            "match_percentage": round((len(matched) / len(job_skills) * 100) if job_skills else 0, 1)
        }
    }

print("OK: Skills matching function defined!")


## 7. Flask Application - API Endpoints

### Flask Architecture

Flask creates a web application with several **routes** (endpoints):

- `GET /` : Displays the main page
- `POST /api/parse-pdf` : Parses a PDF/TXT file
- `POST /api/extract-skills` : Extracts skills
- `POST /api/match-skills` : Matches skills
- `POST /api/optimize-cv` : Optimizes CV
- `POST /api/generate-letter` : Generates letter

### Data Flow

```
Frontend (JavaScript) 
    ↓ (HTTP POST request)
Flask Route Handler
    ↓ (calls function)
Utility module (cv_optimizer, etc.)
    ↓ (calls OpenAI)
OpenAI API
    ↓ (returns result)
Utility module
    ↓ (returns dict)
Flask Route Handler
    ↓ (jsonify)
Frontend (receives JSON)
```


In [ ]:
# Example Flask structure (commented code for notebook)
# To run the full application, use app.py

"""
from flask import Flask, render_template, request, jsonify
from flask_cors import CORS
import os

app = Flask(__name__)
CORS(app)  # Allows cross-origin requests from frontend

# Configuration
UPLOAD_FOLDER = 'uploads'
MAX_FILE_SIZE = 10 * 1024 * 1024  # 10MB
ALLOWED_EXTENSIONS = {'pdf', 'txt'}

os.makedirs(UPLOAD_FOLDER, exist_ok=True)

# Main route - Displays HTML page
@app.route('/')
def index():
    return render_template('index.html')

# API route - Parse PDF
@app.route('/api/parse-pdf', methods=['POST'])
def api_parse_pdf():
    # 1. Check that a file was sent
    if 'file' not in request.files:
        return jsonify({'error': 'No file provided'}), 400
    
    file = request.files['file']
    
    # 2. Check file type
    if not allowed_file(file.filename):
        return jsonify({'error': 'Invalid file type'}), 400
    
    # 3. Read content
    file_content = file.read()
    
    # 4. Extract text
    if file.filename.endswith('.pdf'):
        text = extract_text_from_pdf(file_content)
    else:
        text = file_content.decode('utf-8')
    
    # 5. Return result as JSON
    return jsonify({
        'text': text,
        'filename': file.filename,
        'word_count': len(text.split())
    })

# API route - Optimize CV
@app.route('/api/optimize-cv', methods=['POST'])
def api_optimize_cv():
    # 1. Get JSON data
    data = request.json
    
    # 2. Extract parameters
    cv_text = data.get('cv_text', '')
    job_description = data.get('job_description', '')
    api_key = data.get('api_key', '')
    # ... other parameters
    
    # 3. Validate data
    if not api_key:
        return jsonify({'error': 'API key is required'}), 400
    
    # 4. Call optimization function
    result = optimize_cv(
        cv_text=cv_text,
        job_description=job_description,
        api_key=api_key,
        # ... other parameters
    )
    
    # 5. Handle errors
    if 'error' in result:
        return jsonify({'error': result['error']}), 500
    
    # 6. Return result
    return jsonify(result)

# Run application
if __name__ == '__main__':
    app.run(debug=True, host='127.0.0.1', port=5000)
"""

## 8. Usage Examples

Here's how to use the functions in the code:

### Example 1: Optimize a CV

```python
# Input data
cv_text = "Your CV text..."
job_description = "Job description..."
api_key = "sk-..."

# Call function
result = optimize_cv(
    cv_text=cv_text,
    job_description=job_description,
    api_key=api_key,
    language="fr",
    model="gpt-4o-mini",
    temperature=0.3
)

# Check result
if 'error' in result:
    print(f"Error: {result['error']}")
else:
    print(f"Optimized CV ({result['word_count']} words)")
    print(result['optimized_cv'])
```

### Example 2: Extract and Match Skills

```python
# Extract skills from CV
cv_skills_result = extract_skills(cv_text, api_key, "cv")
cv_skills = cv_skills_result['skills']

# Extract skills from job offer
job_skills_result = extract_skills(job_description, api_key, "job")
job_skills = job_skills_result['skills']

# Match skills
match_result = match_skills(
    cv_skills=cv_skills,
    job_skills=job_skills,
    cv_text=cv_text,
    job_text=job_description,
    api_key=api_key
)

# Display statistics
stats = match_result['stats']
print(f"Match: {stats['match_percentage']}%")
print(f"Matched skills: {len(match_result['matched'])}")
print(f"Missing skills: {len(match_result['job_only'])}")
```


## 9. Key Takeaways

### 🔑 Important Concepts

1. **LangChain**: Framework that simplifies LLM usage
   - `ChatOpenAI`: Wrapper for OpenAI
   - `ChatPromptTemplate`: Structures prompts
   - `chain = prompt | llm`: Creates a processing chain

2. **Temperature**:
   - **Low (0.2-0.3)**: Deterministic, consistent results → Ideal for CVs
   - **High (0.7-1.0)**: More creative, varied → Ideal for letters

3. **Error handling**: Always parse OpenAI errors for clear messages

4. **PDF extraction**: Dual method (pdfplumber + PyPDF2) for robustness

5. **Intelligent matching**: Uses AI to identify "interesting" skills

### 🎯 Best Practices

- ✅ Always validate inputs before calling the API
- ✅ Handle errors with clear user messages
- ✅ Use appropriate temperatures according to use case
- ✅ Normalize data before comparison (lowercase, strip)
- ✅ Clean AI responses (markdown, JSON parsing)

---

**To run the full application:**
```bash
python app.py
```

**Then open:** `http://127.0.0.1:5000`
